In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import re
import pandas as pd
# import optuna.integration.lightgbm as lgb  # 调参用
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score, f1_score
from sklearn.model_selection import KFold, StratifiedKFold

import math
from collections import defaultdict
from gensim.models import Word2Vec
from tqdm import tqdm
import os

import joblib
import random

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas(desc='pandas bar')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 300)
pd.set_option('float_format', lambda x: '%.3f' % x)

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8)

tgt_market = 't1'
data_dir = f'./DATA/{tgt_market}/'
neg_num = 20

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
train = pd.read_table(data_dir+'train.tsv')
train_5core = pd.read_table(data_dir+'train_5core.tsv')

train_cross = train_5core.merge(train, on=['userId', 'itemId'], how='left')
train_5core = train_cross[train_cross['rating_y'].isnull()][['userId', 'itemId']]

train = pd.concat([train, train_5core, train_5core, train_5core, train_5core, train_5core], ignore_index=True)
print(train.shape)

(68418, 3)


In [3]:
train.head()

,userId,itemId,rating
0,t1U1006129,P1016853,5.000
1,t1U1002135,P1019605,5.000
2,t1U1006710,P1032538,5.000
3,t1U1007418,P1012632,5.000
4,t1U1008062,P1022775,5.000


# 提取用户侧和物品侧的特征

In [4]:
# 转成列表
user_feature = train.groupby("userId").agg(
    item_list=("itemId", list),
    rating_list=("rating", list)
).reset_index()

item_feature = train.groupby("itemId").agg(
    user_list=("userId", list),
    rating_list=("rating", list)
).reset_index()

In [5]:
# tfidf特征
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

emb_size = 32
# user_feature['item_list_str'] = user_feature['item_list'].apply(lambda x:' '.join(x))

# tfidf = TfidfVectorizer(ngram_range=(1,1))
# user_tfidf = tfidf.fit_transform(user_feature['item_list_str'])
# # pca = PCA(n_components=emb_size) # 初始化PCA
# svd = TruncatedSVD(n_components=emb_size, random_state=2021)

# # user_tfidf_pca = pca.fit_transform(user_tfidf.toarray())
# user_tfidf_svd = svd.fit_transform(user_tfidf) # 返回降维后的数据

# for i in range(emb_size):
#     user_feature['user_tfidf_svd_emb_{}'.format(i)] = user_tfidf_svd[:, i]
#     # user_feature['user_tfidf_pca_emb_{}'.format(i)] = user_tfidf_pca[:, i]
    
# user_feature = user_feature.drop(['item_list_str'],axis=1)

In [6]:
# 提取user侧和item侧的统计特征


# 用户侧特征
user_feature['item_num'] = user_feature['item_list'].apply(len)
user_feature['item_nuique_num'] = user_feature['item_list'].apply(lambda x:len(set(x)))

# user_feature['rating_sum_u'] = user_feature['rating_list'].apply(sum)
# user_feature['rating_mean_u'] = user_feature['rating_list'].apply(np.mean)
# user_feature['rating_std_u'] = user_feature['rating_list'].apply(np.std)

# 商品侧特征user
item_feature['user_num'] = item_feature['user_list'].apply(len)
item_feature['user_nuique_num'] = item_feature['user_list'].apply(lambda x:len(set(x)))

# item_feature['rating_sum_i'] = item_feature['rating_list'].apply(sum)
# item_feature['rating_mean_i'] = item_feature['rating_list'].apply(np.mean)
# item_feature['rating_std_i'] = item_feature['rating_list'].apply(np.std)

# # 使用item/user交互过的user/item的特征序列的统计值代表item/user的特征
user_feature = user_feature.drop(['item_list','rating_list'],axis=1)
item_feature = item_feature.drop(['user_list','rating_list'],axis=1)

train = train.merge(user_feature, on='userId',how='left')
train = train.merge(item_feature, on='itemId',how='left')

user_feature_col = user_feature.columns
# 用户的商品序列特征，会把所有的特征翻倍
for col in [col for col in item_feature.columns if col not in ['itemId']]:
    user_by_item_tmp = train.groupby(['userId'],as_index=False)[col].agg({f'{col}_max':'max',
                                                                          f'{col}_min':'min',
                                                                          f'{col}_mean':'mean',
                                                                          f'{col}_std':np.std,})
    user_feature = user_feature.merge(user_by_item_tmp,on='userId',how='left')  

# 商品的用户序列特征
for col in [col for col in user_feature_col if col not in ['userId']+['user_tfidf_svd_emb_{}'.format(i) for i in range(emb_size)]]:
    item_by_user_tmp = train.groupby(['itemId'],as_index=False)[col].agg({f'{col}_max':'max',
                                                                          f'{col}_min':'min',
                                                                          f'{col}_mean':'mean',
                                                                          f'{col}_std':np.std,})
    item_feature = item_feature.merge(item_by_user_tmp,on='itemId',how='left')

In [7]:
item_feature['user_num'].mean()
user_feature['item_num'].mean()

6.872727272727273

In [8]:
# word2vec特征
def emb(df, f1, f2, mode='agg'):
    emb_size = 16
    tmp = df.groupby(f1, as_index=False)[f2].agg({'{}_{}_list'.format(f1, f2): list})
    sentences = tmp['{}_{}_list'.format(f1, f2)].values.tolist()
    del tmp['{}_{}_list'.format(f1, f2)]
    for i in range(len(sentences)):
        sentences[i] = [str(x) for x in sentences[i]]

    if os.path.exists(f'./w2v_{tgt_market}.model'):
        print('find w2v model')
        model = Word2Vec.load(f'./w2v_{tgt_market}.model')
    else:
        print('train w2v model')
        model = Word2Vec(sentences, size=emb_size, window=50, min_count=5, sg=0, hs=0, seed=1, iter=5, workers=8)
        model.save(f'./w2v_{tgt_market}.model')
        

    if mode=='agg':
        emb_matrix = []
        for seq in sentences:
            vec = []
            for w in seq:
                if w in model.wv.vocab:
                    vec.append(model.wv[w])
            if len(vec) > 0:
                emb_matrix.append(np.mean(vec, axis=0))
            else:
                emb_matrix.append([0] * emb_size)
        emb_matrix = np.array(emb_matrix)
        for i in range(emb_size):
            tmp['{}_{}_emb_{}'.format(f1, f2, i)] = emb_matrix[:, i]
        
    else:
        itemId2vec = {}
        for itemId in model.wv.vocab:
            itemId2vec[itemId] = model.wv[itemId]
        tmp = pd.DataFrame(columns=[f2])
        tmp[f2] = list(itemId2vec.keys())
        emb_matrix = np.array(list(itemId2vec.values()))
        for i in range(16):
            tmp['{}_emb_{}'.format(f2, i)] = emb_matrix[:, i]
    
    return tmp

# user和item均可做agg_emb和single_emb
user_embed = emb(train.copy(), 'userId', 'itemId', mode='agg')
# user_embed_single = single_emb(train.copy(), 'itemId', 'userId')

# item_embed_agg = agg_emb(train.copy(), 'itemId', 'userId')
item_embed = emb(train.copy(), 'userId', 'itemId', mode='single')

# user_embed = user_embed_agg.merge(user_embed_single, on='userId', how='left')
# item_embed = item_embed_agg.merge(item_embed_single, on='itemId', how='left')

find w2v model
find w2v model


# 构建训练集和测试集

## 构建训练集，未交互过的样本当作负样本

In [9]:
# # 获取正样本
# train_pos = train[['userId','itemId']]
# train_pos['label'] = 1

# # 获取负样本
# item_pool = set(item_feature['itemId'])
# by_userid_group = train_pos.groupby("userId")['itemId']
# user_df_ids = []
# neg_item_df_ids = []
# for userid, group_frame in tqdm(by_userid_group):
#     pos_itemids = set(group_frame.values.tolist())   # 交互过的都是正样本
#     neg_itemids = list(item_pool - pos_itemids)      # 没有交互过的都是负样本
#     neg_item_df_ids += random.sample(neg_itemids, min(neg_num, len(neg_itemids)))
#     user_df_ids += [userid]*min(neg_num, len(neg_itemids))

# train_neg = pd.DataFrame(columns=['userId','itemId'])
# train_neg['userId'] = user_df_ids
# train_neg['itemId'] = neg_item_df_ids
# train_neg['label'] = 0
# train = train_pos.append(train_neg)

# # 合并特征
# train = train.merge(user_feature, on='userId',how='left')
# train = train.merge(item_feature, on='itemId',how='left')
# print(train.shape)

In [10]:
# train.head()

## 构建验证集&测试集

In [11]:
def load_market_valid_run(valid_run_file):  # 把一行item_id分别拆到user_id中,构成<user_id,item_id>pair
    users, items = [], []
    with open(valid_run_file, 'r') as f:
        for line in f:
            linetoks = line.split('\t')
            user_id = linetoks[0]
            item_ids = linetoks[1].strip().split(',')
            for cindex, item_id in enumerate(item_ids):
                users.append(user_id)
                items.append(item_id)

    return users, items

# 验证集，同时也作为训练集
user_ids, item_ids = load_market_valid_run(data_dir+'valid_run.tsv')
valid = pd.DataFrame(columns=['userId','itemId'])
valid['userId'] = user_ids
valid['itemId'] = item_ids

# 合并统计特征
valid = valid.merge(user_feature, on='userId',how='left')
valid = valid.merge(item_feature, on='itemId',how='left')
# 合并w2v特征
valid = valid.merge(user_embed, on='userId',how='left')
valid = valid.merge(item_embed, on='itemId',how='left')

# 测试集
user_ids, item_ids = load_market_valid_run(data_dir+'test_run.tsv')
test = pd.DataFrame(columns=['userId','itemId'])
test['userId'] = user_ids
test['itemId'] = item_ids
# 合并统计特征
test = test.merge(user_feature, on='userId',how='left')
test = test.merge(item_feature, on='itemId',how='left')
# 合并w2v特征
test = test.merge(user_embed, on='userId',how='left')
test = test.merge(item_embed, on='itemId',how='left')
# test.isnull().sum()

In [12]:
# item_CF特征，获取当前item与用于交互过的物品相似度的最大值，最小值，均值，方差等特征
def item_cf(df, user_col, item_col):  # train, 'itemId', 'userId'
    user_item_ = df.groupby(user_col)[item_col].agg(list).reset_index()     # user的item列表
    user_item_dict = dict(zip(user_item_[user_col], user_item_[item_col]))  # 变成字典
    
    sim_item = {}                                  # 里面存的又是字典
    item_cnt = defaultdict(int)  
    for user, items in tqdm(user_item_dict.items()):     # 这段逻辑是用户交互过的item之间的相似度为  1 / math.log(1 + len(items))
        for item in items:                                                  
            item_cnt[item] += 1                    # item出现的频次
            sim_item.setdefault(item, {})          # 查找item键值，不存在设为空字典
            for relate_item in items:  
                if item == relate_item:            # item自身无相似度
                    continue
                
                sim_item[item].setdefault(relate_item, 0)   # 如果不存在，先设为0
                sim_item[item][relate_item] += 1 / math.log(1 + len(items))
                
    sim_item_corr = sim_item.copy()
    for i, related_items in tqdm(sim_item.items()):     # 做个类似于归一化的计算
        for j, cij in related_items.items():  
            sim_item_corr[i][j] = cij / math.sqrt(item_cnt[i]*item_cnt[j])   # 相似度矩阵
  
    return sim_item_corr, user_item_

sim_item_corr, user_item_list = item_cf(train.copy(), 'userId', 'itemId')
# 返回待预测item与当前用户交互过的item的相似度列表
def get_sim_list(cf_data):  # 相似度矩阵, user交互的item列表, 用户id

    userId = cf_data['userId']
    itemId = cf_data['itemId']
    interacted_items = cf_data['itemId_list']   # 可能为空
    sim_score_list = []
    try:
        for i in interacted_items:
            try:
                sim_score_list.append(sim_item_corr[itemId][i])
            except:
                sim_score_list.append(0)
    except:
        sim_score_list.append(0)

    return sim_score_list         # 将预测的item与用户交互过的item的相似度列表返回

# 返回用户交互过的item两两之间的相似度列表
def get_innner_sim_list(itemId_list):  # 相似度矩阵, user交互的item列表, 用户id

    innner_sim_score_list = []
    for i in range(len(itemId_list)):
        for j in range(i+1, len(itemId_list)):
            try:
                innner_sim_score_list.append(sim_item_corr[itemId_list[i]][itemId_list[j]])
            except:
                innner_sim_score_list.append(0)

    return innner_sim_score_list         # 将预测的item与用户交互过的item的相似度列表返回
# 验证集cf
valid_cf = valid[['userId', 'itemId']]
valid_cf = valid_cf.merge(user_item_list.rename({'itemId':'itemId_list'},axis=1), on='userId', how='left')
valid_cf['sim_list'] = valid_cf.parallel_apply(lambda x:get_sim_list(x), axis=1)
valid_cf['sim_mean'] = valid_cf['sim_list'].parallel_apply(np.mean)
valid_cf['sim_max'] = valid_cf['sim_list'].parallel_apply(np.max)
valid_cf['sim_min'] = valid_cf['sim_list'].parallel_apply(np.min)

valid_cf['inner_sim_list'] = valid_cf['itemId_list'].parallel_apply(lambda x:get_innner_sim_list(x))
# valid_cf['inner_sim_mean'] = valid_cf['inner_sim_list'].parallel_apply(np.mean)
# valid_cf['inner_sim_max'] = valid_cf['inner_sim_list'].parallel_apply(np.max)
# valid_cf['inner_sim_min'] = valid_cf['inner_sim_list'].parallel_apply(np.min)
# valid_cf['inner_sim_std'] = valid_cf['inner_sim_list'].parallel_apply(np.std)

valid_cf = valid_cf.drop(['itemId_list', 'sim_list', 'inner_sim_list'],axis=1)
# 测试集cf
test_cf = test[['userId', 'itemId']]
test_cf = test_cf.merge(user_item_list.rename({'itemId':'itemId_list'},axis=1), on='userId', how='left')
test_cf['sim_list'] = test_cf.parallel_apply(lambda x:get_sim_list(x), axis=1)
test_cf['sim_mean'] = test_cf['sim_list'].parallel_apply(np.mean)
test_cf['sim_max'] = test_cf['sim_list'].parallel_apply(np.max)
test_cf['sim_min'] = test_cf['sim_list'].parallel_apply(np.min)

test_cf['inner_sim_list'] = test_cf['itemId_list'].parallel_apply(lambda x:get_innner_sim_list(x))
# test_cf['inner_sim_mean'] = test_cf['inner_sim_list'].parallel_apply(np.mean)
# test_cf['inner_sim_max'] = test_cf['inner_sim_list'].parallel_apply(np.max)
# test_cf['inner_sim_min'] = test_cf['inner_sim_list'].parallel_apply(np.min)
# test_cf['inner_sim_std'] = test_cf['inner_sim_list'].parallel_apply(np.std)


test_cf = test_cf.drop(['itemId_list', 'sim_list', 'inner_sim_list'],axis=1)

100%|██████████| 3543/3543 [00:00<00:00, 18752.71it/s]


In [13]:
# 合并item_cf特征
valid = valid.merge(valid_cf, how='left', on=['userId','itemId'])
test = test.merge(test_cf, how='left', on=['userId','itemId'])

In [14]:
# 将valid的label merge上去
valid_qrel = pd.read_table(data_dir+'valid_qrel.tsv')
valid = valid.merge(valid_qrel, how='left', on=['userId','itemId']).rename({'rating':'label'}, axis=1)
valid['label'] = valid['label'].fillna(0)
print(valid.shape)
# valid.isnull().sum()

(269700, 58)


In [15]:
# lgb模型
useless_cols = ['userId','itemId','label']
def train_model_lgb(data_, test_, y_, folds_, cat_cols=None):
    oof_preds = np.zeros(data_.shape[0])       # 验证集预测结果
    sub_preds = np.zeros(test_.shape[0])       # 测试集预测结果
    feature_importance_df = pd.DataFrame()
    feats = [f for f in data_.columns if f not in useless_cols]
   
    
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_, y_)):
        
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]   # 训练集数据
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]   # 验证集数据
       
        clf = LGBMClassifier(
            n_estimators=4000,   # 4000
            learning_rate=0.08,  # 0.08 
            num_leaves=2**5,
            colsample_bytree=0.8, # 0.8
            subsample=0.9,        # 0.9
            max_depth=5, 
            reg_alpha=.3,    # 0.3
            reg_lambda=.3,   # 0.3
            min_split_gain=.01,
            min_child_weight=2,
            silent=-1,
            verbose=-1,
            n_jobs=8,
        )
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                eval_metric='auc', verbose=300, early_stopping_rounds=100,  # 这个参数有点小，可以再大一点
                # categorical_feature = cat_cols
               )
        oof_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]   # 验证集结果
        
        
        sub_preds += clf.predict_proba(test_[feats], num_iteration=clf.best_iteration_)[:, 1] / folds_.n_splits  # 测试集结果
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
    
    print('=====Full AUC score %.6f=====' % roc_auc_score(y_, oof_preds))
    
    test_['score'] = sub_preds
    data_['score'] = oof_preds  # 验证集结果
    
    # valid_['score'] = val_preds

    # return oof_preds, test_[['userId', 'itemId', 'score']], valid_[['userId', 'itemId', 'score']], feature_importance_df
    return data_[['userId', 'itemId', 'score']], test_[['userId', 'itemId', 'score']], feature_importance_df

# 训练&预测

In [16]:
train = valid
y = train['label']
folds = KFold(n_splits=3, shuffle=True, random_state=546789)
s_folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=546789)
oof_preds, test_preds, importances = train_model_lgb(train, test, y, s_folds)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	training's auc: 0.969576	training's binary_logloss: 0.0203851	valid_1's auc: 0.919815	valid_1's binary_logloss: 0.0251003
Fold  1 AUC : 0.919815
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[92]	training's auc: 0.974096	training's binary_logloss: 0.0197679	valid_1's auc: 0.928031	valid_1's binary_logloss: 0.0242165
Fold  2 AUC : 0.928031
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[89]	training's auc: 0.973714	training's binary_logloss: 0.020193	valid_1's auc: 0.925066	valid_1's binary_logloss: 0.0239806
Fold  3 AUC : 0.925066
=====Full AUC score 0.924240=====


In [17]:
run_dir = './baseline_outputs/sample_run/'

In [18]:
test_preds.sort_values(by=['userId', 'score'], ascending=[True, False], inplace=True)
oof_preds.sort_values(by=['userId', 'score'], ascending=[True, False], inplace=True)

test_preds.to_csv(run_dir+f'{tgt_market}/test_pred.tsv', sep='\t', index=False)
oof_preds.to_csv(run_dir+f'{tgt_market}/valid_pred.tsv', sep='\t', index=False)